In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='../data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='../data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

model = LogisticRegression(input_size, num_classes)

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()    # The loss function for classify 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

Files already downloaded


In [4]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):# Nice way to get iterating counts and the data you want.
        images = Variable(images.view(-1, 28*28))      # Get a 100 x 784 matrix, every row contains 28x28 = 784 numbers. 
        labels = Variable(labels)                      # Get a 100 x  1  matrix, with 0~9 in every row
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))   # Get a 100 x 784 matrix
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)   # Output.data > get numbers in output
    #Returns the maximum value of each row of the input Tensor in the given dimension dim.  > First return
    #＊＊＊Also returns the index location of each maximum value found.                      > Second return
    total += labels.size(0)                     # To know how many lables in total.
    correct += (predicted == labels).sum()      # Count correct prediction.
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Epoch: [1/5], Step: [100/600], Loss: 2.1775
Epoch: [1/5], Step: [200/600], Loss: 2.0936
Epoch: [1/5], Step: [300/600], Loss: 1.9679
Epoch: [1/5], Step: [400/600], Loss: 1.9093
Epoch: [1/5], Step: [500/600], Loss: 1.8866
Epoch: [1/5], Step: [600/600], Loss: 1.8149
Epoch: [2/5], Step: [100/600], Loss: 1.6539
Epoch: [2/5], Step: [200/600], Loss: 1.7051
Epoch: [2/5], Step: [300/600], Loss: 1.5585
Epoch: [2/5], Step: [400/600], Loss: 1.5482
Epoch: [2/5], Step: [500/600], Loss: 1.4168
Epoch: [2/5], Step: [600/600], Loss: 1.4819
Epoch: [3/5], Step: [100/600], Loss: 1.4792
Epoch: [3/5], Step: [200/600], Loss: 1.3251
Epoch: [3/5], Step: [300/600], Loss: 1.1989
Epoch: [3/5], Step: [400/600], Loss: 1.3185
Epoch: [3/5], Step: [500/600], Loss: 1.3318
Epoch: [3/5], Step: [600/600], Loss: 1.2686
Epoch: [4/5], Step: [100/600], Loss: 1.2063
Epoch: [4/5], Step: [200/600], Loss: 1.1406
Epoch: [4/5], Step: [300/600], Loss: 1.1577
Epoch: [4/5], Step: [400/600], Loss: 1.1316
Epoch: [4/5], Step: [500/600], L

In [ ]:
# Save the Model
torch.save(model.state_dict(), 'model.pkl')